# Preparing dataframes for pseudobulking analysis
## Using raw data of DC (use object prepared for CytoTRACE --> NR_unint_DC_cytotrace.h5ad), need to prepare two types of dataframes: 
* sum by sample, so for each DC subset (3 dataframes) made a dataframe that has genes as rows and integration_id (samples) as columns
* then make a metadata dataframe that has two columns --> integration_id and cancer type

## Here just doing on primary tumours

In [1]:
#load packages
import scanpy as sc
import os
import pandas as pd

In [2]:
#change current directory 
os.chdir('/scratch/user/s4436039/scdata/Pseudobulk')
os.getcwd()

'/scratch/user/s4436039/scdata/Pseudobulk'

In [3]:
# Read in the data 
data = sc.read_h5ad('/scratch/user/s4436039/scdata/CytoTRACE_R/NR_unint_DC_cytotrace.h5ad')

In [4]:
data

AnnData object with n_obs × n_vars = 30241 × 15074
    obs: 'nCount_RNA', 'nFeature_RNA', 'sample_type', 'cancer_type', 'patient_id', 'sample_id', 'percent.mt', 'site', 'sample_type_major', 'cancer_subtype', 'integration_id', 'ident', 'dataset_id', 'NR_annotations_simple', 'sample_type_major2', 'cancer_with_H', 'cancer_broadest'
    var: 'name'

In [5]:
# filter to just primary tumours 
data = data[data.obs["sample_type_major2"].isin(["primary tumour"])]

In [6]:
data

View of AnnData object with n_obs × n_vars = 17585 × 15074
    obs: 'nCount_RNA', 'nFeature_RNA', 'sample_type', 'cancer_type', 'patient_id', 'sample_id', 'percent.mt', 'site', 'sample_type_major', 'cancer_subtype', 'integration_id', 'ident', 'dataset_id', 'NR_annotations_simple', 'sample_type_major2', 'cancer_with_H', 'cancer_broadest'
    var: 'name'

# Make pseudobulk dataframes: 

In [14]:
#subset out each DC
data_DC1 = data[data.obs["NR_annotations_simple"].isin(["cDC1"])]
data_DC2 = data[data.obs["NR_annotations_simple"].isin(["cDC2"])]
data_mregDC = data[data.obs["NR_annotations_simple"].isin(["mregDC"])]

In [17]:
#checking how many samples in each (this will have to match metadata dataframe below)
num_unique_integration_ids = data_mregDC.obs['integration_id'].nunique()
print(f"Number of unique integration_id values: {num_unique_integration_ids}")

Number of unique integration_id values: 333


In [18]:
data_DC1_matrix = data_DC1.X.toarray()
data_DC2_matrix = data_DC2.X.toarray()
data_mregDC_matrix = data_mregDC.X.toarray()

In [19]:
genes_DC1 = data_DC1.var_names
obs_DC1 = data_DC1.obs

genes_DC2 = data_DC2.var_names
obs_DC2 = data_DC2.obs

genes_mregDC = data_mregDC.var_names
obs_mregDC = data_mregDC.obs

In [16]:
# Create a DataFrame with cells as rows and genes as columns
data_DC1_df = pd.DataFrame(data_DC1_matrix, index=obs_DC1.index, columns=genes_DC1)
data_DC2_df = pd.DataFrame(data_DC2_matrix, index=obs_DC2.index, columns=genes_DC2)
data_mregDC_df = pd.DataFrame(data_mregDC_matrix, index=obs_mregDC.index, columns=genes_mregDC)

In [17]:
# Add integration_id
data_DC1_df['integration_id'] = obs_DC1['integration_id']
data_DC2_df['integration_id'] = obs_DC2['integration_id']
data_mregDC_df['integration_id'] = obs_mregDC['integration_id']

In [18]:
data_DC1_df.head()

,FAM87B,LINC00115,FAM41C,SAMD11,NOC2L,KLHL17,PLEKHN1,HES4,ISG15,AGRN,...,MT-ATP8,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,integration_id
GSE215120_AM1_ACCAGTAAGACTGGGT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,13.0,29.0,5.0,0.0,11.0,5.0,1.0,15.0,GSE215120_Acral_MEL_AM1
GSE215120_AM1_ACGTCAACAAGGACTG-1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,27.0,0.0,...,1.0,52.0,162.0,22.0,0.0,64.0,10.0,1.0,74.0,GSE215120_Acral_MEL_AM1
GSE215120_AM1_ATAACGCGTAGCGCTC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,11.0,35.0,8.0,0.0,14.0,5.0,0.0,20.0,GSE215120_Acral_MEL_AM1
GSE215120_AM1_ATAGACCCATTACCTT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,17.0,38.0,12.0,0.0,20.0,4.0,0.0,25.0,GSE215120_Acral_MEL_AM1
GSE215120_AM1_ATTGGTGAGTCTCGGC-1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,0.0,...,0.0,23.0,50.0,13.0,1.0,29.0,6.0,1.0,29.0,GSE215120_Acral_MEL_AM1


In [19]:
# Group by integration_id and sum to pseudobulk
pseudobulk_DC1_df = data_DC1_df.groupby('integration_id').sum().transpose()
pseudobulk_DC2_df = data_DC2_df.groupby('integration_id').sum().transpose()
pseudobulk_mregDC_df = data_mregDC_df.groupby('integration_id').sum().transpose()

/scratch/temp/12182602/ipykernel_1321129/1587600822.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_DC1_df = data_DC1_df.groupby('integration_id').sum().transpose()
/scratch/temp/12182602/ipykernel_1321129/1587600822.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_DC2_df = data_DC2_df.groupby('integration_id').sum().transpose()
/scratch/temp/12182602/ipykernel_1321129/1587600822.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the fut

In [20]:
pseudobulk_DC1_df.head()

integration_id,GGSE223063_GLIO_1,GGSE223063_GLIO_3,GSE112271_HCC_Pt13,GSE112271_HCC_Pt14,GSE131907_LUAD_T0006,GSE131907_LUAD_T0008,GSE131907_LUAD_T0009,GSE131907_LUAD_T0018,GSE131907_LUAD_T0019,GSE131907_LUAD_T0020,...,PMID32561858_breast_50,PMID32561858_breast_51,PMID32561858_breast_52,PRJCA005422_HGSOC1_PT,PRJCA005422_HGSOC2_PT,PRJCA005422_HGSOC3_PT,PRJCA005422_HGSOC4_PT,PRJCA005422_HGSOC7_PT,PRJCA005422_HGSOC8_PT,PRJCA005422_HGSOC9_PT
FAM87B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LINC00115,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
FAM41C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SAMD11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
NOC2L,0.0,0.0,0.0,2.0,1.0,0.0,7.0,1.0,2.0,1.0,...,0.0,5.0,0.0,3.0,1.0,2.0,3.0,2.0,4.0,1.0


In [21]:
pseudobulk_DC2_df.head()

integration_id,GGSE223063_GLIO_1,GGSE223063_GLIO_3,GSE112271_HCC_Pt13,GSE112271_HCC_Pt14,GSE131907_LUAD_T0006,GSE131907_LUAD_T0008,GSE131907_LUAD_T0009,GSE131907_LUAD_T0018,GSE131907_LUAD_T0019,GSE131907_LUAD_T0020,...,PMID32561858_breast_54,PRJCA005422_HGSOC1_PT,PRJCA005422_HGSOC2_PT,PRJCA005422_HGSOC3_PT,PRJCA005422_HGSOC4_PT,PRJCA005422_HGSOC6_PT,PRJCA005422_HGSOC7_PT,PRJCA005422_HGSOC8_PT,PRJCA005422_HGSOC9_PT,PRJCA005422_HGSOC10_PT
FAM87B,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LINC00115,0.0,0.0,0.0,1.0,3.0,0.0,2.0,0.0,3.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
FAM41C,1.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
SAMD11,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
NOC2L,4.0,0.0,3.0,6.0,17.0,6.0,7.0,5.0,10.0,17.0,...,0.0,22.0,8.0,7.0,6.0,3.0,7.0,0.0,2.0,3.0


In [22]:
pseudobulk_mregDC_df.head()

integration_id,GGSE223063_GLIO_1,GSE112271_HCC_Pt13,GSE112271_HCC_Pt14,GSE131907_LUAD_T0006,GSE131907_LUAD_T0008,GSE131907_LUAD_T0009,GSE131907_LUAD_T0018,GSE131907_LUAD_T0019,GSE131907_LUAD_T0020,GSE131907_LUAD_T0025,...,PMID32561858_breast_51,PRJCA005422_HGSOC1_PT,PRJCA005422_HGSOC2_PT,PRJCA005422_HGSOC3_PT,PRJCA005422_HGSOC4_PT,PRJCA005422_HGSOC6_PT,PRJCA005422_HGSOC7_PT,PRJCA005422_HGSOC8_PT,PRJCA005422_HGSOC9_PT,PRJCA005422_HGSOC10_PT
FAM87B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LINC00115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
FAM41C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SAMD11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NOC2L,2.0,0.0,3.0,3.0,5.0,0.0,2.0,5.0,3.0,4.0,...,4.0,1.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0


In [23]:
# save as csv
pseudobulk_DC1_df.to_csv("pseudobulk_DC1_data.csv", index=True)
pseudobulk_DC2_df.to_csv("pseudobulk_DC2_data.csv", index=True)
pseudobulk_mregDC_df.to_csv("pseudobulk_mregDC_data.csv", index=True)

# Make metadata dataframes

In [21]:
data_DC1

View of AnnData object with n_obs × n_vars = 3582 × 15074
    obs: 'nCount_RNA', 'nFeature_RNA', 'sample_type', 'cancer_type', 'patient_id', 'sample_id', 'percent.mt', 'site', 'sample_type_major', 'cancer_subtype', 'integration_id', 'ident', 'dataset_id', 'NR_annotations_simple', 'sample_type_major2', 'cancer_with_H', 'cancer_broadest'
    var: 'name'

In [22]:
metadata_DC1 = data_DC1.obs[['integration_id', 'cancer_broadest']].groupby('integration_id').first()
metadata_DC2 = data_DC2.obs[['integration_id', 'cancer_broadest']].groupby('integration_id').first()
metadata_mregDC = data_mregDC.obs[['integration_id', 'cancer_broadest']].groupby('integration_id').first()

/scratch/temp/12205626/ipykernel_876409/2617498612.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  metadata_DC1 = data_DC1.obs[['integration_id', 'cancer_broadest']].groupby('integration_id').first()
/scratch/temp/12205626/ipykernel_876409/2617498612.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  metadata_DC2 = data_DC2.obs[['integration_id', 'cancer_broadest']].groupby('integration_id').first()
/scratch/temp/12205626/ipykernel_876409/2617498612.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cur

In [25]:
metadata_mregDC.head()

,cancer_broadest
integration_id,
GGSE223063_GLIO_1,GBM
GSE112271_HCC_Pt13,HCC
GSE112271_HCC_Pt14,HCC
GSE131907_LUAD_T0006,NSCLC
GSE131907_LUAD_T0008,NSCLC


In [29]:
metadata_mregDC.shape

(333, 1)

In [30]:
# set the index (integration_id) to be a column
metadata_DC1 = metadata_DC1.reset_index()
metadata_DC2 = metadata_DC2.reset_index()
metadata_mregDC = metadata_mregDC.reset_index()

In [33]:
metadata_mregDC.head()

,integration_id,cancer_broadest
0,GGSE223063_GLIO_1,GBM
1,GSE112271_HCC_Pt13,HCC
2,GSE112271_HCC_Pt14,HCC
3,GSE131907_LUAD_T0006,NSCLC
4,GSE131907_LUAD_T0008,NSCLC


In [34]:
# save as csv
metadata_DC1.to_csv("pseudobulk_metadata_DC1.csv", index=False)
metadata_DC2.to_csv("pseudobulk_metadata_DC2.csv", index=False)
metadata_mregDC.to_csv("pseudobulk_metadata_mregDC.csv", index=False)